In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
#import the necessary libraries
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import keras
import tensorflow as tf


In [3]:
#define the batch size for training
batch_size =  32 
#define the number of epoch for training
epochs = 10 
#define the dimensionality of the encoding process
latent_dim = 128 
#define the number of samples to train on
num_samples = 1000

In [4]:
#define the file location
data_path = "/content/drive/MyDrive/Colab Notebooks/vie.txt"
#define an empty list to store the words/sentences in English
input_texts = []
#define an empty list to store the characters in Spanish
target_texts = []
#define a set to store the words/sentences in English
#a set data type and not a list is used to avoid repetition of characters
input_characters = set()
#define a set to store the characters in Spanish
target_characters = set()
#read the data file and parse by each line
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

for line in lines[: min(num_samples, len(lines) - 1)]:
    #split each line into the input text, target text and the other unnecessary text
    input_text, target_text, _ = line.split('\t')
    #we use tab as the start sequence character for the target, and \n as end sequence character
    target_text = '\t' + target_text + '\n'
    #append the English words for each text to the empty list
    input_texts.append(input_text)
    #append the Spanish words for each text to the empty list
    target_texts.append(target_text)
    #select the letters or words in English
    for char in input_text:
        #check if the letter is not in the list
        if char not in input_characters:
            #append the letter that is not on the list
            input_characters.add(char)
    #select the letters or words in Spanish
    for char in target_text:
        #check if the letter or word is not in the list
        if char not in target_characters:
            #append the letter or word that is not on the list
            target_characters.add(char)

In [5]:
print(input_characters)

{'Y', 'T', '3', 'p', 'g', 'C', '!', '6', 'l', '$', 't', 'N', '1', 'P', 'o', 'k', 'W', 'i', 'v', 'M', 'A', 'F', 'S', 'e', 'D', 'n', 'E', '2', 'z', 'L', ' ', '0', '-', 'H', 'I', 'c', 'B', 'q', "'", 'a', 'V', 'j', '?', 'K', 'm', 'd', 'u', 'b', 'O', 'U', ',', 'h', 'r', '.', 'f', 'w', 'R', 'G', 'y', 'J', ':', 's', 'x'}


In [6]:
print(target_characters)

{'ự', 'p', 'ỗ', 'ò', 't', 'ở', 'ỡ', ')', 'v', 'A', 'ấ', 'ẫ', 'ụ', 'Ô', 'è', 'ó', 'Á', 'c', 'đ', 'q', 'ọ', 'V', 'ỏ', 'á', 'ầ', 'ề', 'ê', 'ệ', '.', 'ú', 'ă', 'ỹ', 'G', 'ớ', 'ũ', 'C', 'ã', 'o', '\n', 'k', 'ặ', 'i', 'Ă', 'â', 'D', 'ô', 'n', 'ờ', 'ễ', '2', '\t', '0', 'ộ', 'B', 'j', 'ẽ', 'm', 'ử', 'ố', 'U', 'ể', 'ậ', 'y', 'x', 'T', 'ằ', '3', 'í', '6', 'ù', '1', 'ẵ', 'N', 'ạ', 'F', 'S', 'Q', 'e', 'ợ', 'L', 'é', 'I', 'X', 'ữ', 'ẩ', 'ư', 'ắ', 'ì', 'a', 'Đ', 'K', 'ị', 'd', 'b', 'ồ', ',', 'r', 'ẻ', 'ỉ', 'Ở', 'O', 'R', 's', 'ĩ', 'Ý', 'à', 'g', '!', 'l', 'ý', 'P', 'ả', 'ủ', 'M', 'ỳ', 'Ờ', 'E', '(', 'ẹ', 'z', ' ', 'H', 'õ', 'ế', '?', 'ứ', 'u', 'h', 'ơ', 'ẳ', 'ừ', 'ổ'}


In [7]:
print(input_texts[: 50])

['Run!', 'Help!', 'Go on.', 'Hello!', 'Hurry!', 'I try.', 'I try.', 'Eat it.', 'Eat it.', 'Got it!', 'Help me!', 'I agree.', 'Perfect!', 'We know.', 'We know.', 'You run.', 'Cheer up.', 'He tries.', 'He tries.', 'Hurry up.', 'I forgot.', "I'll try.", "I'll try.", "I'm bald.", "I'm busy.", 'Too late.', 'Come here.', 'Come here.', 'Come here.', 'Go to bed.', 'I hate TV.', 'I laughed.', 'I laughed.', 'I will go.', 'Is it bad?', "It's cold.", "It's ours.", "It's ours.", 'Keep back.', 'Keep back.', 'Look here.', 'She cried.', 'She cried.', 'Sit there.', 'Stay home.', "What's up?", "What's up?", 'Are you OK?', 'Find a job.', 'Hurry home.']


In [8]:
print(target_texts[:50])

['\tChạy!\n', '\tGiúp tôi với!\n', '\tTiếp tục đi.\n', '\tChào bạn.\n', '\tNhanh lên nào!\n', '\tTôi sẽ thử.\n', '\tTôi sẽ thử xem sao.\n', '\tĂn đi.\n', '\tĂn nó đi.\n', '\tHiểu rồi!\n', '\tCứu tôi với!\n', '\tTôi cũng nghĩ như vậy.\n', '\tHoàn hảo!\n', '\tChúng tôi biết.\n', '\tChúng ta biết.\n', '\tBạn chạy.\n', '\tĐừng có rầu rĩ quá như thế.\n', '\tHắn thử.\n', '\tAnh thử.\n', '\tThoáng cái chân lên!\n', '\tTôi quên mất rồi.\n', '\tTôi sẽ thử.\n', '\tTôi sẽ thử xem sao.\n', '\tTôi bị hói\n', '\tTôi đang bận.\n', '\tMuộn quá.\n', '\tLại đây nào.\n', '\tLại đây.\n', '\tLại đây đi.\n', '\tĐi ngủ đi.\n', '\tTôi ghét ti vi.\n', '\tTôi đã cười.\n', '\tTôi cười.\n', '\tTôi sẽ đi.\n', '\tNó có tồi không?\n', '\tLạnh.\n', '\tĐó là của chúng tôi.\n', '\tĐó là của chúng ta.\n', '\tLùi lại.\n', '\tLùi lại đi.\n', '\tLại đây.\n', '\tNó đã khóc.\n', '\tCô ấy đã khóc\n', '\tHãy ngồi ở đó.\n', '\tỞ nhà.\n', '\tGì thế?\n', '\tCó chuyện gì?\n', '\tBạn có sao không?\n', '\tHãy tìm một công việc đi.\n

In [9]:
#define the sorted list of English words
input_characters = sorted(list(input_characters))
#define the sorted list of Spanish words
target_characters = sorted(list(target_characters))
#define the number of English words
num_encoder_tokens = len(input_characters)
#define the number of Spanish words
num_decoder_tokens = len(target_characters)
#define the maximum length of the English sentences
max_encoder_seq_length = max([len(txt) for txt in input_texts])
#define the maximum length of the Spanish sentences
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [10]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs", max_decoder_seq_length)

Number of samples: 1000
Number of unique input tokens: 63
Number of unique output tokens: 132
Max sequence length for inputs: 19
Max sequence length for outputs 47


In [11]:
#index each characters in English
input_token_index = dict(
[(char, i) for i, char in enumerate(input_characters)])
#index each characters in Spanish
target_token_index = dict(
[(char, i) for i, char in enumerate(target_characters)])

In [12]:
#define the input data of the encoder as a 3-dimensional matrix populated with zeros
#the shape of the matrix is the length of the input text by the length of the max encoder by the number of encoder characters
#the np.zeros() function takes an argument of the specified dtype. Here we use float32
encoder_input_data = np.zeros(
(len(input_texts), max_encoder_seq_length, num_encoder_tokens
), dtype='float32')
#define the input data of the decoder as a 3-dimensional matrix populated with zeros
decoder_input_data = np.zeros(
(len(input_texts), max_decoder_seq_length, num_decoder_tokens
), dtype='float32')
#define the target data of the decoder as a 3-dimensional matrix populated with zeros
decoder_target_data = np.zeros(
(len(input_texts), max_decoder_seq_length, num_decoder_tokens
), dtype='float32')

In [13]:
#parse the input and output texts
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        #define that the decoder_target_data is one time step ahead of the decoder_input_data by 
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # define the decoder_target_data to be ahead by one timestep and will not include the first character
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [14]:
class GlobalAttention(keras.layers.Layer):
    def __init__(self,latent_dim):
        super(GlobalAttention,self).__init__()
        self.latent_dim=latent_dim
        self.K=Dense(units=self.latent_dim)
        self.Q=Dense(units=self.latent_dim)
    def call(self,encoder_outputs,S):
        q=self.Q(S)
        k=self.K(encoder_outputs)
        alpha=tf.nn.softmax(tf.nn.tanh(tf.matmul(k,tf.expand_dims(q,axis=-1))),axis=1)
        contex=tf.reduce_sum(alpha*encoder_outputs,axis=1)
        return contex

In [45]:
class LSTMwithAttention(keras.layers.Layer):
    def __init__(self,latent_dim):
        super(LSTMwithAttention,self).__init__()
        self.latent_dim=latent_dim
        self.contex_layer=GlobalAttention(self.latent_dim)
        self.decoder_lstm = LSTM(self.latent_dim, return_sequences=True, return_state=True)
        self.cat=keras.layers.Concatenate(axis=1)
        
    def call(self,decoder_inputs,encoder_outputs, encoder_states, max_decoder_seq_length):
        state_s=encoder_states[0]
        output=[]
        for i in range(max_decoder_seq_length):
            contex=self.contex_layer(encoder_outputs, state_s)
            x=decoder_inputs[:,i:i+1,:]
            x_contex=tf.concat([tf.expand_dims(contex,axis=1), x], axis=-1)
            decoder_outputs, state_h, state_c =self.decoder_lstm(x_contex,initial_state=encoder_states)
            state_s=state_h
            encoder_states=[state_h, state_c]
            output.append(decoder_outputs)
        return [self.cat(output),state_h, state_c]

In [33]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True,return_sequences=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

In [46]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm=LSTMwithAttention(latent_dim)
decoder_lstm_outputs,_,_=decoder_lstm(decoder_inputs,encoder_outputs, encoder_states, max_decoder_seq_length)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

In [47]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [48]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [50]:
model.fit([encoder_input_data[:1000], decoder_input_data[:1000]], decoder_target_data[:1000],
         batch_size=batch_size,
         epochs=20,
         validation_split=0.2)

Epoch 1/20
25/25 [==============================] - 6s 237ms/step - loss: 0.8887 - accuracy: 0.7728 - val_loss: 1.0771 - val_accuracy: 0.7293
Epoch 2/20
25/25 [==============================] - 6s 235ms/step - loss: 0.8691 - accuracy: 0.7766 - val_loss: 1.0607 - val_accuracy: 0.7311
Epoch 3/20
25/25 [==============================] - 6s 231ms/step - loss: 0.8513 - accuracy: 0.7800 - val_loss: 1.0420 - val_accuracy: 0.7376
Epoch 4/20
25/25 [==============================] - 6s 229ms/step - loss: 0.8349 - accuracy: 0.7837 - val_loss: 1.0258 - val_accuracy: 0.7396
Epoch 5/20
25/25 [==============================] - 6s 233ms/step - loss: 0.8199 - accuracy: 0.7859 - val_loss: 1.0138 - val_accuracy: 0.7410
Epoch 6/20
25/25 [==============================] - 6s 235ms/step - loss: 0.8073 - accuracy: 0.7883 - val_loss: 0.9987 - val_accuracy: 0.7417
Epoch 7/20
25/25 [==============================] - 6s 230ms/step - loss: 0.7947 - accuracy: 0.7898 - val_loss: 0.9870 - val_accuracy: 0.7453
Epoch 

In [51]:
model.save("/content/drive/MyDrive/Colab Notebooks/attention model")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/attention model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/attention model/assets


In [52]:
encoder_model = Model(encoder_inputs, [encoder_outputs, state_h, state_c])

In [53]:
# Reverse-lookup token index to decode sequence back to something readable
reverse_input_char_index = dict(
(i, char) for char, i in input_token_index.items()
)
reverse_target_char_index = dict(
(i, char) for char, i in target_token_index.items()
)

In [71]:
def decode_sequence(input_seq):
    encoder_outputs_, state_h_, state_c_ = encoder_model(input_seq)
    states_value = [state_h_, state_c_]
    
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_lstm, h, c = decoder_lstm(target_seq, encoder_outputs_, states_value, 1)
        output_tokens = decoder_dense(output_lstm)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        if (len(decoded_sentence) > max_decoder_seq_length or sampled_char == '\n'):
            stop_condition = True
            
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        
        states_value = [h, c]
    
    return decoded_sentence

In [73]:
for seq_index in range(120,140):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print()
    print(f"Input sentence: {input_texts[seq_index]}")
    print(f"Decoded sentence: {decoded_sentence}")


Input sentence: Don't be shy.
Decoded sentence: Đừng đi nha cha thông?


Input sentence: Don't be shy.
Decoded sentence: Đừng đi nha cha thông?


Input sentence: Drive faster.
Decoded sentence: Hãy đi thôi đi.


Input sentence: Good evening.
Decoded sentence: Hãy đi thôi đi.


Input sentence: Hang on, Tom.
Decoded sentence: Bạn có thi thi ch thông?


Input sentence: I admire Tom.
Decoded sentence: Tôi đã đã đi.


Input sentence: I am at home.
Decoded sentence: Tôi đã thể chác  hông đây.


Input sentence: I am curious.
Decoded sentence: Tôi đã thể chể nhười đi.


Input sentence: I don't know.
Decoded sentence: Tôi đã đi nhười đi.


Input sentence: I don't know.
Decoded sentence: Tôi đã đi nhười đi.


Input sentence: I hear music.
Decoded sentence: Tôi ch nh nh th.


Input sentence: I like bread.
Decoded sentence: Tôi chích thích nhác đây.


Input sentence: I like bread.
Decoded sentence: Tôi chích thích nhác đây.


Input sentence: I won't lose!
Decoded sentence: Tôi đã đi nhười đi.


I